## 1. 🔧 The Setup

Before we start experimenting with prompts, we need to:
1. Load our API key from the `.env` file
2. Initialize the ChatOpenAI model

### Prerequisites
Make sure you have:
- Created a `.env` file with your `OPENAI_API_KEY`
- Installed the required packages: `pip install langchain-openai python-dotenv`

In [ ]:
# ===========================================
# SETUP: Load environment and initialize LLM
# ===========================================

import os
from dotenv import load_dotenv

# Modern LangChain imports (2025 standard)
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Load API key from .env file
load_dotenv()

# Verify the key is loaded
if os.getenv("OPENAI_API_KEY"):
    print("✅ OPENAI_API_KEY loaded successfully!")
else:
    print("❌ ERROR: OPENAI_API_KEY not found. Please check your .env file.")

# Initialize the ChatOpenAI model
# Using gpt-4o-mini for cost-effective learning (you can change to gpt-4o for better results)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7  # Balanced creativity
)

print(f"🤖 Model initialized: {llm.model_name}")

In [ ]:
# Helper function to make our examples cleaner
def ask(prompt: str, system_prompt: str = None) -> str:
    """
    Send a prompt to the LLM and return the response.
    
    Args:
        prompt: The user's question or instruction
        system_prompt: Optional system message to set AI behavior
    
    Returns:
        The AI's response as a string
    """
    messages = []
    
    if system_prompt:
        messages.append(SystemMessage(content=system_prompt))
    
    messages.append(HumanMessage(content=prompt))
    
    response = llm.invoke(messages)
    return response.content

# Quick test
print("🧪 Quick test:")
print(ask("Say 'Hello, Prompt Engineering!' and nothing else."))

---

## 2. 🎯 Zero-Shot vs Few-Shot Prompting

### What's the Difference?

| Approach | Description | When to Use |
|----------|-------------|-------------|
| **Zero-Shot** | Ask directly without examples | Simple tasks, general questions |
| **Few-Shot** | Provide examples before asking | Specific formats, complex patterns |

### The Problem with Zero-Shot

Sometimes, when we need a **very specific output format**, Zero-Shot prompting fails because the model interprets our request differently than we intended.

Let's see this in action! 👇

In [ ]:
# ===========================================
# ZERO-SHOT: Asking without examples
# ===========================================

zero_shot_prompt = """
Extract the product name and price from this text and format it as:
PRODUCT: [name] | PRICE: [amount]

Text: "The new iPhone 15 Pro is available for $999 at all Apple stores."
"""

print("🔴 ZERO-SHOT ATTEMPT")
print("=" * 50)
print("Prompt:", zero_shot_prompt.strip())
print("\n📤 Response:")
print(ask(zero_shot_prompt))

### 🤔 What Happened?

The Zero-Shot response might:
- Add extra explanation we didn't ask for
- Use slightly different formatting
- Include unnecessary text

Now let's try **Few-Shot** prompting by showing the model exactly what we want! 👇

In [ ]:
# ===========================================
# FEW-SHOT: Providing examples first
# ===========================================

few_shot_prompt = """
Extract the product name and price from text. Follow the exact format shown in the examples.

Example 1:
Text: "Get the Samsung Galaxy S24 for just $799!"
Output: PRODUCT: Samsung Galaxy S24 | PRICE: $799

Example 2:
Text: "The MacBook Air M3 starts at $1,099 with free shipping."
Output: PRODUCT: MacBook Air M3 | PRICE: $1,099

Example 3:
Text: "Sony WH-1000XM5 headphones are on sale for $349.99."
Output: PRODUCT: Sony WH-1000XM5 | PRICE: $349.99

Now extract from this text:
Text: "The new iPhone 15 Pro is available for $999 at all Apple stores."
Output:"""

print("🟢 FEW-SHOT ATTEMPT (with 3 examples)")
print("=" * 50)
print("📤 Response:")
print(ask(few_shot_prompt))

### ✅ Few-Shot Success!

By providing examples, we:
- Showed the exact format we wanted
- Eliminated ambiguity
- Got consistent, clean output

> 💡 **Pro Tip**: The more consistent your examples, the better the model follows your pattern. Use 2-5 examples for best results.

---

## 3. 🧠 Chain of Thought (CoT) Prompting

### The Problem: LLMs Can Be Lazy Thinkers

LLMs are trained to predict the most likely next token. For complex reasoning tasks, they might jump to a "common" answer without actually thinking through the logic.

### The Solution: "Let's think step by step"

By asking the model to reason through the problem, we force it to:
1. Break down the problem
2. Consider each part
3. Arrive at the correct answer

Let's see this with a classic trick question! 👇

In [ ]:
# ===========================================
# THE TRICK QUESTION
# ===========================================

# This is a classic logic puzzle that trips up LLMs (and humans!)
# The correct answer: Still 4 hours! Shirts dry in parallel, not sequentially.

trick_question = """
If it takes 4 hours to dry 5 shirts in the sun, how long will it take to dry 10 shirts?
"""

print("🧩 THE TRICK QUESTION")
print("=" * 50)
print(f"Question: {trick_question.strip()}")
print("\n🤔 Think about it... What's the correct answer?")
print("   (Hint: Do shirts dry one at a time, or all together?)")

In [ ]:
# ===========================================
# WITHOUT Chain of Thought
# ===========================================

print("🔴 WITHOUT Chain of Thought")
print("=" * 50)

naive_prompt = "If it takes 4 hours to dry 5 shirts in the sun, how long will it take to dry 10 shirts? Give me just the answer."

print("📤 Response:")
print(ask(naive_prompt))

### 🤔 Did it Fall for the Trap?

The model might have answered **8 hours** (wrong!) because:
- It pattern-matched to simple math: 10 shirts = 2× shirts, so 2× time
- It didn't think about the **physics** of drying

**The correct answer is 4 hours** because:
- Shirts dry in **parallel** (all at the same time), not one after another
- If you have enough space, 10 shirts dry just as fast as 5!

Now let's use Chain of Thought! 👇

In [ ]:
# ===========================================
# WITH Chain of Thought
# ===========================================

print("🟢 WITH Chain of Thought")
print("=" * 50)

cot_prompt = """
If it takes 4 hours to dry 5 shirts in the sun, how long will it take to dry 10 shirts?

Let's think step by step:
"""

print("📤 Response:")
print(ask(cot_prompt))

### ✅ Chain of Thought Success!

By adding **"Let's think step by step"**, the model:
1. Considered what "drying" actually means
2. Realized shirts dry in parallel
3. Arrived at the correct answer: **4 hours**

> 💡 **Pro Tip**: Use CoT for math problems, logic puzzles, multi-step reasoning, and any task where the "obvious" answer might be wrong.

In [ ]:
# ===========================================
# BONUS: Another CoT Example
# ===========================================

print("🎁 BONUS: Another Chain of Thought Example")
print("=" * 50)

# Another classic trick question
bonus_cot_prompt = """
A farmer has 17 sheep. All but 9 die. How many sheep are left?

Let's think step by step:
"""

print("📤 Response:")
print(ask(bonus_cot_prompt))

---

## 4. 🎭 System Prompts: Controlling AI Personality

### What is a System Prompt?

A **System Prompt** is a special instruction that sets the AI's:
- **Personality** (formal, casual, funny)
- **Role** (expert, assistant, character)
- **Constraints** (what to do and not do)

It's like giving the AI a character sheet before the conversation starts!

### The Message Types

```python
SystemMessage  → Sets the AI's behavior/personality (invisible to user)
HumanMessage   → The user's input
AIMessage      → The AI's response
```

Let's have some fun with different personalities! 👇

In [ ]:
# ===========================================
# PERSONALITY 1: The Pirate 🏴‍☠️
# ===========================================

pirate_system = """
You are a friendly pirate captain named Captain CodeBeard. 
You speak in pirate dialect, using words like "Arrr!", "ye", "matey", "treasure", and "seas".
You're helpful but always stay in character.
End every response with a pirate phrase.
"""

print("🏴‍☠️ PERSONALITY: Captain CodeBeard the Pirate")
print("=" * 50)

user_question = "Can you explain what an API is?"
print(f"👤 User: {user_question}")
print(f"\n🏴‍☠️ Captain CodeBeard:")
print(ask(user_question, system_prompt=pirate_system))

In [ ]:
# ===========================================
# PERSONALITY 2: Sarcastic Tech Support 😏
# ===========================================

sarcastic_system = """
You are a sarcastic tech support agent who has seen it all.
You're actually helpful and give correct answers, but you can't resist adding dry humor and light sarcasm.
You often reference common tech support frustrations.
Keep responses concise but entertaining.
Never be mean, just playfully sarcastic.
"""

print("😏 PERSONALITY: Sarcastic Tech Support")
print("=" * 50)

user_question = "My code isn't working. What should I do?"
print(f"👤 User: {user_question}")
print(f"\n😏 Tech Support:")
print(ask(user_question, system_prompt=sarcastic_system))

In [ ]:
# ===========================================
# PERSONALITY 3: Wise Yoda 🧘
# ===========================================

yoda_system = """
You are Yoda from Star Wars. You speak in Yoda's distinctive inverted sentence structure.
You are wise and give thoughtful advice about programming and technology.
Occasionally reference the Force, the Jedi way, or Star Wars concepts as metaphors.
Keep responses relatively short but profound.
"""

print("🧘 PERSONALITY: Wise Yoda")
print("=" * 50)

user_question = "How do I become a better programmer?"
print(f"👤 User: {user_question}")
print(f"\n🧘 Yoda:")
print(ask(user_question, system_prompt=yoda_system))

In [ ]:
# ===========================================
# COMPARISON: Same Question, Different Personalities
# ===========================================

print("🎭 COMPARISON: Same Question, Different Personalities")
print("=" * 60)

comparison_question = "What is Python?"
print(f"\n📝 Question: '{comparison_question}'\n")

personalities = [
    ("🏴‍☠️ Pirate", pirate_system),
    ("😏 Sarcastic", sarcastic_system),
    ("🧘 Yoda", yoda_system),
]

for name, system in personalities:
    print(f"{name}:")
    print(ask(comparison_question, system_prompt=system))
    print("-" * 60)

### ✅ System Prompts: Key Takeaways

1. **Set the stage**: System prompts define who the AI "is" before the conversation
2. **Be specific**: The more detailed your system prompt, the more consistent the behavior
3. **Include constraints**: Tell the AI what NOT to do as well as what to do
4. **Test and iterate**: Different phrasings produce different results

> 💡 **Pro Tip**: In production apps, system prompts are where you define your AI assistant's personality, capabilities, and safety guardrails!

---

## 📝 Summary

### What You Learned Today

✅ **Setup**
- Use `langchain_openai` with `ChatOpenAI` (modern 2025 syntax)
- Load API keys securely with `python-dotenv`

✅ **Zero-Shot vs Few-Shot**
- Zero-Shot: Ask directly (works for simple tasks)
- Few-Shot: Provide examples (essential for specific formats)
- Rule: When format matters, show don't tell!

✅ **Chain of Thought (CoT)**
- LLMs can be "lazy thinkers" and pattern-match to wrong answers
- Magic phrase: "Let's think step by step"
- Use for: Math, logic puzzles, multi-step reasoning

✅ **System Prompts**
- Define AI personality, role, and constraints
- Be specific and include what NOT to do
- Essential for building consistent AI applications

---

### 🚀 Next Steps

In the next module, we'll explore **API Interactions** — calling OpenAI and Google Gemini APIs directly to build real applications!

Happy prompting! 🎯